[과제] Online_Retail 데이터셋으로 고객 세그먼테이션을 수행하세요.
- 필요한 데이터 전처리
- 적합한 파생변수 개발(feature engineering) - RFM
- 적합한 고객 그룹 파악(실루엣 계수 시각화 활용)

In [3]:
import pandas as pd
import datetime
import math
import numpy as np
import matplotlib.pyplot as plt

retail_df = pd.read_excel('./dataset/Online_Retail.xlsx')

In [6]:
print(retail_df.shape)
retail_df.head()

(541909, 8)


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [9]:
retail_df.isnull().sum()
# 고객이 null값인 부분 삭제필요

InvoiceNo           0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     135080
Country             0
dtype: int64

In [13]:
retail_df.dropna(subset=['CustomerID'],inplace=True)
retail_df.isnull().sum()

InvoiceNo      0
StockCode      0
Description    0
Quantity       0
InvoiceDate    0
UnitPrice      0
CustomerID     0
Country        0
dtype: int64

In [14]:
retail_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 406829 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    406829 non-null  object        
 1   StockCode    406829 non-null  object        
 2   Description  406829 non-null  object        
 3   Quantity     406829 non-null  int64         
 4   InvoiceDate  406829 non-null  datetime64[ns]
 5   UnitPrice    406829 non-null  float64       
 6   CustomerID   406829 non-null  float64       
 7   Country      406829 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 27.9+ MB


In [17]:
retail_df['Amount']=retail_df.UnitPrice * retail_df.Quantity
retail_df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Amount
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,15.30
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,22.00
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34


In [103]:
retail_df.loc[retail_df.CustomerID == 12503]

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Amount
45622,C540271,M,Manual,-1,2011-01-06 11:51:00,1126.0,12503.0,Spain,-1126.0


In [48]:
Frequency = retail_df.loc[retail_df.InvoiceNo.str.contains('C').isnull()].groupby('CustomerID')['InvoiceNo'].count()

In [77]:
df_1 = pd.DataFrame(retail_df.loc[retail_df.InvoiceNo.str.contains('C').isnull()].groupby('CustomerID')['InvoiceNo'].count())
df_1


,InvoiceNo
CustomerID,
12346.0,1
12347.0,182
12348.0,31
12349.0,73
12350.0,17
...,...
18280.0,10
18281.0,7
18282.0,12


In [79]:
df_2 = pd.DataFrame(retail_df.loc[retail_df.InvoiceNo.str.contains('C').notnull()].groupby('CustomerID')['InvoiceNo'].count())
df_2

,InvoiceNo
CustomerID,
12346.0,1
12352.0,10
12359.0,6
12362.0,8
12365.0,1
...,...
18272.0,4
18274.0,11
18276.0,2


In [81]:
df_3 = df_1 - df_2

In [83]:
df_3.replace(np.nan ,0,inplace=True)

In [88]:
df_4 = df_1 - df_3
df_4

,InvoiceNo
CustomerID,
12346.0,1.0
12347.0,182.0
12348.0,31.0
12349.0,73.0
12350.0,17.0
...,...
18280.0,10.0
18281.0,7.0
18282.0,1.0


In [100]:
df_4.isnull().sum()

InvoiceNo    33
dtype: int64

In [102]:
df_4[df_4.InvoiceNo.isnull()]

,InvoiceNo
CustomerID,
12503.0,NaN
12505.0,NaN
12605.0,NaN
12666.0,NaN
12870.0,NaN
12943.0,NaN
13154.0,NaN
13693.0,NaN
13829.0,NaN


In [105]:
retail_df.loc[retail_df.CustomerID == 12870.0]

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Amount
18974,C537824,22220,CAKE STAND LOVEBIRD 2 TIER WHITE,-1,2010-12-08 14:27:00,9.95,12870.0,United Kingdom,-9.95
18975,C537824,22784,LANTERN CREAM GAZEBO,-1,2010-12-08 14:27:00,4.95,12870.0,United Kingdom,-4.95


In [75]:
np.where(abc.isnull(), bbb,abc)

ValueError: operands could not be broadcast together with shapes (4372,) (4339,) (4372,) 

In [54]:
Monetaty = retail_df.groupby('CustomerID')['Amount'].sum()
retail_df.groupby('CustomerID')['Amount'].sum()

CustomerID
12346.0       0.00
12347.0    4310.00
12348.0    1797.24
12349.0    1757.55
12350.0     334.40
            ...   
18280.0     180.60
18281.0      80.82
18282.0     176.60
18283.0    2094.88
18287.0    1837.28
Name: Amount, Length: 4372, dtype: float64